## Co-culture interaction experiment | Part 1

<br>
<b>Description</b> : This notebook outlines the procedure by which the a reference set of genes related to `DC<-Tcell` and `Tcell<-DC` interactions were derived from the co-culture data.<br>
<b>Author</b> : Alma Andersson (andera29@gene.com)<br>
<b>Date</b> : 08/14/2024

Import relevant packages

In [3]:
import anndata as ad
import os.path as osp
import numpy as np
import scanpy as sc

In [4]:
DATA_DIR = '../../data/imod'

## Data Processing
The cell culture and meta data was downloaded from here: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE135382. <br>
The files were merged into one anndata object with the meta data in the obs column.

In [ ]:
ad_gt = ad.read_h5ad(osp.join(DATA_DIR,'cell_culture_data','adata.h5ad'))

In [ ]:
ad_gt

Assign more interpretable names to the cells

In [5]:
ct_old = ['CD11c+',
 'TCRb+',
 'TCRb+ CD11c+',
 'Ag+ TCRb+ CD11c+',
 'CD160+ TCRb+',
 'Ag+ CD11c+',
 'Icos+ TCRb+',
 'Tigit+ TCRb+']

ct_new = ['DC',
 'Tcell',
 'TCRb+ CD11c+',
 'Ag+ TCRb+ CD11c+',
 'CD160+ TCRb+',
 'Ag+ CD11c+',
 'Icos+ TCRb+',
 'Tigit+ TCRb+']

ct_map = {old:new for old,new in zip(ct_old,ct_new)}

In [6]:
ad_gt.obs['cell_type'] = ad_gt.obs['sorting_scheme'].map(ct_map)

In [7]:
ad_gt.obs['state'] = ad_gt.obs['treatment'].astype(str) + '_' + ad_gt.obs['cell_type'].astype(str)

Define which cells to keep

In [8]:
keep_labels = ['Co-culture_DC','Co-culture_Tcell','Mono-culture_DC','Mono-culture_Tcell']

In [9]:
keep_idx = np.isin(ad_gt.obs['state'].values,keep_labels)

In [10]:
ad_gt = ad_gt[keep_idx,:].copy()

## Scanpy DE Analysis

Normalize data (for wilcoxon test)

In [ ]:
ad_gt.raw = ad_gt.X.copy()
sc.pp.normalize_total(ad_gt, target_sum = 1e4)
sc.pp.log1p(ad_gt)

Define the groups to compare with scanpy `rank_genes_groups`

In [19]:
groups = [('Co-culture_Tcell','Mono-culture_Tcell'), ('Co-culture_DC','Mono-culture_DC')]

Run DE analysis for each group

In [29]:
results = dict()
for gidx in range(len(groups)):
    sc.tl.rank_genes_groups(ad_gt, groupby = 'state', groups = groups[gidx],method='wilcoxon')
    dedf = sc.get.rank_genes_groups_df(ad_gt, group = groups[gidx][0],)
    results[groups[gidx][0].split('_')[1]] = dedf

Save each list

In [35]:
for key,val in results.items():
    val.to_csv(osp.join(DATA_DIR,'cell_culture_data','de_lists',f'{key}_scanpy.csv'))